In [2]:
import scanpy as sc

In [5]:
adata = sc.read("/orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/K562_essential_raw_singlecell_01.h5ad")

/orcd/data/omarabu/001/opitcho/miniforge3/envs/omnicell/lib/python3.9/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
adata.obs['gene'].unique()

['NAF1', 'BUB1', 'UBL5', 'C9orf16', 'TIMM9', ..., 'RPL7A', 'NUP155', 'FDPS', 'RBM22', 'POLR3A']
Length: 2058
Categories (2058, object): ['AAAS', 'AAMP', 'AARS', 'AARS2', ..., 'ZRSR2', 'ZW10', 'ZWINT', 'non-targeting']

In [6]:
adata.obs['gene'].unique()

['NAF1', 'BUB1', 'UBL5', 'C9orf16', 'TIMM9', ..., 'RPL7A', 'NUP155', 'FDPS', 'RBM22', 'POLR3A']
Length: 2058
Categories (2058, object): ['AAAS', 'AAMP', 'AARS', 'AARS2', ..., 'ZRSR2', 'ZW10', 'ZWINT', 'non-targeting']

In [9]:
type(adata.obs["gene"])

pandas.core.series.Series